In [22]:
import pandas as pd
import paramiko
import json
import numpy as np
from os import listdir

In [24]:
ssh_client = paramiko.SSHClient()
ssh_client.load_system_host_keys()
# ssh_client.connect('srv01775.soton.ac.uk')
# help(ssh_client.connect)
ssh_client.connect('srv01775.soton.ac.uk',username='mp2c18',password='Mon2real')


ssh_client.open_sftp()
sftp_client = ssh_client.open_sftp()

#### results of RELAX using gene-specific trees ####

In [3]:
#### get list of files for HKY ####
path="/home/mp2c18/data/hyphy/relax_spe_tree/GROUP1/HKY"
genes=[]
# remote_file = sftp_client.open('remote_filename')
for file in sftp_client.listdir(path):
    if file[-4:]==".fna":
        genes+=[str(file.split('.')[0][8:])]
print len(genes),genes
df=pd.DataFrame([],columns=['gene','ncodons']+[pfx+sfx for pfx in ['SymvsFL','RuvsFL','GivsFL','GivsRu'] for sfx in ['_k','_pval']])
    
for gene in sorted(set(genes)):
    dic={}
    dic['gene']=gene
#     print gene
    # get data from json files
    for comp in ['SymvsFL','RuvsFL','GivsFL','GivsRu']:
        try:
            remote_file = sftp_client.open(path+'/aligned_'+gene+'_rooted_'+comp+'.RELAX.json')
        except IOError:
            continue
        relax_json = json.load (remote_file)
        ncodons=int(relax_json['input']['number of sites'])
        pval=float(relax_json['test results']['p-value'])
        k=float(relax_json['test results']['relaxation or intensification parameter'])
        dic['ncodons']=ncodons
        dic[comp+'_pval']=pval
        dic[comp+'_k']=k
        remote_file.close()
#         print dic
    if 'GivsFL_k' not in dic.keys():
        print '\t'+path+'/aligned_'+gene+'.fna'+'\\'
    df=df.append(dic,ignore_index=True)
# print df

67 ['Rmag_0012', 'Rmag_0013', 'Rmag_0046', 'Rmag_0052', 'Rmag_0077', 'Rmag_0116', 'Rmag_0117', 'Rmag_0126', 'Rmag_0142', 'Rmag_0146', 'Rmag_0160', 'Rmag_0169', 'Rmag_0178', 'Rmag_0224', 'Rmag_0241', 'Rmag_0267', 'Rmag_0286', 'Rmag_0289', 'Rmag_0302', 'Rmag_0319', 'Rmag_0321', 'Rmag_0329', 'Rmag_0419', 'Rmag_0420', 'Rmag_0434', 'Rmag_0497', 'Rmag_0498', 'Rmag_0534', 'Rmag_0537', 'Rmag_0542', 'Rmag_0547', 'Rmag_0548', 'Rmag_0560', 'Rmag_0570', 'Rmag_0574', 'Rmag_0576', 'Rmag_0588', 'Rmag_0595', 'Rmag_0659', 'Rmag_0668', 'Rmag_0692', 'Rmag_0694', 'Rmag_0699', 'Rmag_0700', 'Rmag_0701', 'Rmag_0704', 'Rmag_0741', 'Rmag_0767', 'Rmag_0774', 'Rmag_0800', 'Rmag_0801', 'Rmag_0806', 'Rmag_0807', 'Rmag_0833', 'Rmag_0853', 'Rmag_0865', 'Rmag_0884', 'Rmag_0915', 'Rmag_0931', 'Rmag_0944', 'Rmag_0962', 'Rmag_0981', 'Rmag_0990', 'Rmag_1012', 'Rmag_1027', 'Rmag_1038', 'Rmag_1071']
	/home/mp2c18/data/hyphy/relax_spe_tree/GROUP1/HKY/aligned_Rmag_0807.fna\


In [4]:
#### get list of files for GTR ####
path="/home/mp2c18/data/hyphy/relax_spe_tree/GROUP1"
genes2=[]
# remote_file = sftp_client.open('remote_filename')
for file in sftp_client.listdir(path):
#     if file[-13:]=="Ru.RELAX.json":
#         genes2+=[str(file.split('.')[0][8:-7])]
    if file[-4:]==".fna":
        genes2+=[str(file.split('.')[0][8:])]

print len(genes2),genes2[:10]

df2=df.copy()

for gene in sorted(set(genes2)):
    dic={}
    dic['gene']=gene

    # get data from json files
    for comp in ['SymvsFL','RuvsFL','GivsFL','GivsRu']:
        try:
            remote_file = sftp_client.open(path+'/aligned_'+gene+'_rooted_'+comp+'.RELAX.json')
        except IOError:
            continue
        relax_json = json.load (remote_file)
        ncodons=int(relax_json['input']['number of sites'])
        pval=float(relax_json['test results']['p-value'])
        k=float(relax_json['test results']['relaxation or intensification parameter'])

        dic['ncodons']=ncodons
        dic[comp+'_pval']=pval
        dic[comp+'_k']=k
        remote_file.close()
    if 'GivsFL_k' not in dic.keys():
        print '\t'+path+'/aligned_'+gene+'.fna'+'\\'
    df2=df2.append(dic,ignore_index=True)


650 ['Rmag_0001', 'Rmag_0002', 'Rmag_0003', 'Rmag_0004', 'Rmag_0005', 'Rmag_0006', 'Rmag_0007', 'Rmag_0008', 'Rmag_0009', 'Rmag_0010']
	/home/mp2c18/data/hyphy/relax_spe_tree/GROUP1/aligned_Rmag_0205.fna\


In [5]:
print len(df2[df2.isnull().any(axis=1)])
df2[df2.isnull().any(axis=1)]


4


,gene,ncodons,SymvsFL_k,SymvsFL_pval,RuvsFL_k,RuvsFL_pval,GivsFL_k,GivsFL_pval,GivsRu_k,GivsRu_pval
21,Rmag_0329,322,0.151017,0.004779,NaN,NaN,3.306422e-01,0.00045,1.219522,0.892046
52,Rmag_0807,148,0.009195,0.004298,0.019503,0.000219,NaN,NaN,1.402507,0.148178
195,Rmag_0205,198,0.038055,0.003599,0.023343,0.001833,NaN,NaN,1.374601,0.074342
550,Rmag_0824,377,NaN,NaN,0.007900,0.000145,5.421011e-20,0.00068,1.215863,0.188315


In [6]:
df3=df2.copy()
i=2
# while i<len(list(df3.iloc[:,2:])):
#     print df3.iloc[:,i:i+2]
    

#     i+=2
NS='NS'
# NS=1
NS=np.nan
df3.loc[df3['SymvsFL_pval'] > 0.05, 'SymvsFL_k'] = NS
df3.loc[df3['RuvsFL_pval'] > 0.05, 'RuvsFL_k'] = NS
df3.loc[df3['GivsFL_pval'] > 0.05, 'GivsFL_k'] = NS
df3.loc[df3['GivsRu_pval'] > 0.05, 'GivsRu_k'] = NS
# for item in zip(sorted(df3['SymvsFL_k']),[type(item) for item in sorted(df3['SymvsFL_k'])]):
#     print item
df3[(1 < df3['RuvsFL_k']) & (df3['RuvsFL_k'] < df3['GivsFL_k'])]
# print df3[(1 > df3['RuvsFL_k'])]
pd.set_option('display.max_rows', 500)

temp=df3[['gene','ncodons','SymvsFL_k','RuvsFL_k','GivsFL_k','GivsRu_k']]

temp.to_csv('/Users/maeperez/Desktop/relax/relax_results_spe_tree.txt',sep='\t',header=True,index=False)


print temp[['gene','ncodons','SymvsFL_k','RuvsFL_k','GivsFL_k','GivsRu_k']].describe()
temp[['gene','SymvsFL_k','RuvsFL_k','GivsFL_k','GivsRu_k']].info()

          SymvsFL_k      RuvsFL_k      GivsFL_k      GivsRu_k
count  3.490000e+02  3.370000e+02  3.770000e+02  1.450000e+02
mean   1.197457e+00  6.628913e-01  7.875687e-01  1.918704e+00
std    3.569097e+00  2.634278e+00  3.031869e+00  3.982422e+00
min    3.388132e-21  1.694066e-21  2.646978e-23  3.605099e-13
25%    9.594997e-02  1.296046e-01  1.207116e-01  2.872130e-01
50%    3.871059e-01  3.782580e-01  4.093016e-01  1.128532e+00
75%    5.676925e-01  5.383313e-01  5.631747e-01  1.861350e+00
max    3.391780e+01  3.698717e+01  4.558330e+01  3.782885e+01
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 5 columns):
gene         717 non-null object
SymvsFL_k    349 non-null float64
RuvsFL_k     337 non-null float64
GivsFL_k     377 non-null float64
GivsRu_k     145 non-null float64
dtypes: float64(4), object(1)
memory usage: 28.1+ KB


In [55]:
#### get list of files for 1 tree: screwed up ####
path="/home/mp2c18/data/hyphy/relax_1_tree/GROUP1"
genes2=[]
# remote_file = sftp_client.open('remote_filename')
for file in sftp_client.listdir(path):
#     if file[-13:]=="Ru.RELAX.json":
#         genes2+=[str(file.split('.')[0][8:-7])]
    if file[-6:]==".relax":
        if len(str(file.split('.')[0][8:]).split('_'))==2:
            genes2+=[str(file.split('.')[0][8:])]

print len(genes2),genes2[:10]

df2=pd.DataFrame([],columns=['gene','ncodons']+[pfx+sfx for pfx in ['SymvsFL','RuvsFL','GivsFL','GivsRu'] for sfx in ['_k','_pval']])
diccomp={'GIGAS vs FREE-LIVING':'GivsFL', 'RUTHIA vs FREE-LIVING':'RuvsFL','GIGAS vs RUTHIA':'GivsRu','Symb vs FREE-LIVING':'SymvsFL'}
for gene in set(genes2):
    dic={}
    dic['gene']=gene
#     print gene
    remote_file = sftp_client.open(path+'/aligned_'+gene+'.relax')
    lines=remote_file.read().splitlines()
    remote_file.close()
    for line in lines:
        if line in diccomp.keys():
            comp=diccomp[line]
            continue
        if line[:59] == '>Loaded a multiple sequence alignment with **13** sequences':
            dic['ncodons']= int(line.split(',')[1].split('**')[1])
            continue
        if line[:44]=='* Relaxation/intensification parameter (K) =':
            dic[comp+'_k']=float(line.split()[-1])
            continue
        if line[:27]=='Likelihood ratio test **p =':
            dic[comp+'_pval']=float(line.split('**')[1].split()[-1])   
    df2=df2.append(dic,ignore_index=True)
print len(df2[df2.isnull().any(axis=1)])
df2[df2.isnull().any(axis=1)]

715 ['Rmag_0052', 'Rmag_0767', 'Rmag_0741', 'Rmag_0012', 'Rmag_0701', 'Rmag_0853', 'Rmag_0169', 'Rmag_0800', 'Rmag_0990', 'Rmag_0534']
8


,gene,ncodons,SymvsFL_k,SymvsFL_pval,RuvsFL_k,RuvsFL_pval,GivsFL_k,GivsFL_pval,GivsRu_k,GivsRu_pval
56,Rmag_0936,154,0.19,0.0004,0.40,0.0260,0.46,0.0076,NaN,NaN
106,Rmag_0275,230,NaN,NaN,1.00,1.0000,0.67,0.4456,0.05,0.0001
161,Rmag_0996,256,0.04,0.0012,0.08,0.0018,NaN,NaN,2.33,0.0089
229,Rmag_0323,191,7.07,0.1608,13.61,0.1190,NaN,NaN,0.60,0.0404
236,Rmag_0328,240,0.00,0.0000,0.00,0.0001,NaN,NaN,0.64,0.0318
564,Rmag_0832,285,0.43,0.0145,0.41,0.0971,NaN,NaN,0.41,0.0005
580,Rmag_0177,183,0.00,0.0035,0.00,0.0055,NaN,NaN,1.07,0.8450
651,Rmag_0585,407,0.06,0.0026,0.11,0.0095,NaN,NaN,1.28,0.5517


In [34]:
#### get list of files for 1 tree: screwed up ####
path="/home/mp2c18/data/hyphy/relax_spe_tree/redo"
genes2=[]
for file in sftp_client.listdir(path):

    if file[-6:]==".relax":
#         print(str(file.split('.')[0][8:]).split('_'))
        if len(str(file.split('.')[0][8:]).split('_')[:-1])==2:
            genes2+=[str(file.split('.')[0][8:])]

print len(genes2),genes2[:10]

['Rmag', '0043', 'full']
['Rmag', '0045', 'full']
['Rmag', '0146', 'full']
['Rmag', '0502', 'full']
['Rmag', '0514', 'full']
['Rmag', '0566', 'full']
['Rmag', '1053', 'full']
7 ['Rmag_0043_full', 'Rmag_0045_full', 'Rmag_0146_full', 'Rmag_0502_full', 'Rmag_0514_full', 'Rmag_0566_full', 'Rmag_1053_full']


In [35]:
df2=pd.DataFrame([],columns=['gene','ncodons']+[pfx+sfx for pfx in ['SymvsFL','RuvsFL','GivsFL','GivsRu'] for sfx in ['_k','_pval']])
diccomp={'GIGAS vs FREE-LIVING':'GivsFL', 'RUTHIA vs FREE-LIVING':'RuvsFL','GIGAS vs RUTHIA':'GivsRu','Symb vs FREE-LIVING':'SymvsFL'}
for gene in set(genes2):
    dic={}
    dic['gene']=gene
#     print gene
    remote_file = sftp_client.open(path+'/aligned_'+gene+'.relax')
    lines=remote_file.read().splitlines()
    remote_file.close()
    for line in lines:
        if line in diccomp.keys():
            comp=diccomp[line]
            continue
        if line[:59] == '>Loaded a multiple sequence alignment with **13** sequences':
            dic['ncodons']= int(line.split(',')[1].split('**')[1])
            continue
        if line[:44]=='* Relaxation/intensification parameter (K) =':
            dic[comp+'_k']=float(line.split()[-1])
            continue
        if line[:27]=='Likelihood ratio test **p =':
            dic[comp+'_pval']=float(line.split('**')[1].split()[-1])   
    df2=df2.append(dic,ignore_index=True)
print len(df2[df2.isnull().any(axis=1)])
df2[df2.isnull().any(axis=1)]

7


,gene,ncodons,SymvsFL_k,SymvsFL_pval,RuvsFL_k,RuvsFL_pval,GivsFL_k,GivsFL_pval,GivsRu_k,GivsRu_pval
0,Rmag_0514_full,81,NaN,NaN,NaN,NaN,1.00,1.0000,NaN,NaN
1,Rmag_0146_full,398,NaN,NaN,NaN,NaN,0.10,0.0125,NaN,NaN
2,Rmag_0045_full,320,NaN,NaN,NaN,NaN,0.00,0.2744,NaN,NaN
3,Rmag_0043_full,768,NaN,NaN,NaN,NaN,0.35,0.0038,NaN,NaN
4,Rmag_0566_full,415,NaN,NaN,NaN,NaN,1.00,1.0000,NaN,NaN
5,Rmag_1053_full,435,NaN,NaN,NaN,NaN,0.51,0.3235,NaN,NaN
6,Rmag_0502_full,554,NaN,NaN,NaN,NaN,0.45,0.0049,NaN,NaN


In [10]:
from os import listdir
import pandas as pd

#### get list of files for 1 tree: screwed up ####
path="/Users/maeperez/Desktop/HYPHY/redo"
genes2=[]
for file in listdir(path):

    if file[-6:]==".relax":
#         print(str(file.split('.')[0][8:]).split('_'))
        if len(str(file.split('.')[0][8:]).split('_'))==2:
            genes2+=[str(file.split('.')[0][8:])]

print len(genes2),genes2[:10]

df2=pd.DataFrame([],columns=['gene','ncodons']+[pfx+sfx for pfx in ['SymvsFL','RuvsFL','GivsFL','GivsRu'] for sfx in ['_k','_pval']])
diccomp={'GIGAS vs FREE-LIVING':'GivsFL', 'RUTHIA vs FREE-LIVING':'RuvsFL','GIGAS vs RUTHIA':'GivsRu','Symb vs FREE-LIVING':'SymvsFL'}
for gene in set(genes2):
    dic={}
    dic['gene']=gene
#     print gene
    remote_file = open(path+'/aligned_'+gene+'.relax')
    lines=remote_file.read().splitlines()
    remote_file.close()
    for line in lines:
        if line in diccomp.keys():
            comp=diccomp[line]
            continue
        if line[:59] == '>Loaded a multiple sequence alignment with **13** sequences':
            dic['ncodons']= int(line.split(',')[1].split('**')[1])
            continue
        if line[:44]=='* Relaxation/intensification parameter (K) =':
            dic[comp+'_k']=float(line.split()[-1])
            continue
        if line[:27]=='Likelihood ratio test **p =':
            dic[comp+'_pval']=float(line.split('**')[1].split()[-1])   
    df2=df2.append(dic,ignore_index=True)
print len(df2[df2.isnull().any(axis=1)])
df2[df2.isnull().any(axis=1)]
df2

7 ['Rmag_0045', 'Rmag_0043', 'Rmag_0514', 'Rmag_0146', 'Rmag_0566', 'Rmag_1053', 'Rmag_0502']
1


,gene,ncodons,SymvsFL_k,SymvsFL_pval,RuvsFL_k,RuvsFL_pval,GivsFL_k,GivsFL_pval,GivsRu_k,GivsRu_pval
0,Rmag_0146,398,0.11,0.0000,0.01,0.0000,0.02,0.0010,1.52,0.4141
1,Rmag_0045,320,NaN,NaN,0.03,0.2453,0.00,0.2500,1.57,0.1734
2,Rmag_0043,768,1.99,0.1790,0.00,0.0000,0.17,0.0001,1.00,1.0000
3,Rmag_0514,81,0.20,0.2594,0.31,0.5422,0.34,0.1678,4.68,0.2157
4,Rmag_1053,435,0.09,0.0018,0.47,0.0009,0.46,0.0744,0.00,0.3955
5,Rmag_0566,415,0.65,0.0088,0.58,0.0016,17.39,0.2940,1.88,0.0015
6,Rmag_0502,554,0.15,0.0001,0.50,0.0001,0.54,0.2695,1.04,0.8037
